In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

import collections
import logging
import os
import pathlib
import re
import string
import sys
import time
import math
import tempfile
from functools import partial

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import pandas as pd

from orion.data import load_signal
from orion import Orion
from orion.data import load_anomalies

from mlprimitives.custom.timeseries_preprocessing import time_segments_aggregate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from mlprimitives.custom.timeseries_preprocessing import rolling_window_sequences
from orion.primitives.timeseries_preprocessing import slice_array_by_dims
from mlprimitives import load_primitive
import numpy as np
from orion.primitives.tadgan import TadGAN

from tensorflow.keras import backend as K
from keras.layers.merge import _Merge

from src.configuration.constants import PROCESSED_DATA_DIRECTORY
import pickle

logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

Using TensorFlow backend.


In [2]:
with open(os.path.join(PROCESSED_DATA_DIRECTORY, 'MSL.pickle'), 'rb') as f:
    msl_dataset = pickle.load(f)
    
with open(os.path.join(PROCESSED_DATA_DIRECTORY, 'MSL_test.pickle'), 'rb') as f:
    msl_test_dataset = pickle.load(f)

In [3]:
X_train = msl_dataset['X_train']
y_train = msl_dataset['y_train']
index_train = msl_dataset['index_train']


X_test = msl_test_dataset['X_test']
y_test = msl_test_dataset['y_test']
index_test = msl_test_dataset['index_test']

In [4]:
input_shape=X_train[0].shape
target_shape=y_train[0].shape
latent_dim=20
learning_rate=0.0005
epochs=70
batch_size=64
iterations_critic=5
latent_shape = (latent_dim, 1)

shape = np.asarray(X_train)[0].shape
length = shape[0]
target_shape = np.asarray(y_train)[0].shape


generator_reshape_dim = length // 2
generator_reshape_shape = (length // 2, 1)
encoder_reshape_shape = latent_shape

encoder_input_shape = shape
generator_input_shape = latent_shape
critic_x_input_shape = target_shape
critic_z_input_shape = latent_shape

lstm_units = 100
dense_units = 20

In [7]:
def build_encoder(input_shape, lstm_units, dense_units, encoder_reshape_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=lstm_units, return_sequences=True)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.Reshape(target_shape=encoder_reshape_shape))
    return tf.keras.Model(x, model(x))
              
encoder = build_encoder(
    input_shape=encoder_input_shape,
    lstm_units=lstm_units,
    dense_units=dense_units,
    encoder_reshape_shape=encoder_reshape_shape,
)
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100, 55)]         0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 20, 1)             524820    
Total params: 524,820
Trainable params: 524,820
Non-trainable params: 0
_________________________________________________________________


In [8]:
def build_generator(input_shape, generator_reshape_dim, generator_reshape_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=generator_reshape_dim))
    model.add(tf.keras.layers.Reshape(target_shape=generator_reshape_shape))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(tf.keras.layers.UpSampling1D(size=2))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=1)))
    model.add(tf.keras.layers.Activation(activation='tanh'))
    return tf.keras.Model(x, model(x))
              
generator = build_generator(
    input_shape=generator_input_shape,
    generator_reshape_dim=generator_reshape_dim,
    generator_reshape_shape=generator_reshape_shape,
)
generator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 100, 1)            133787    
Total params: 133,787
Trainable params: 133,787
Non-trainable params: 0
_________________________________________________________________


In [9]:
def build_critic_x(input_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1))
    return tf.keras.Model(x, model(x))
              
critic_x = build_critic_x(
    input_shape=critic_x_input_shape
)
critic_x.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 100, 1)]          0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 1)                 67393     
Total params: 67,393
Trainable params: 67,393
Non-trainable params: 0
_________________________________________________________________


In [10]:
def build_critic_z(input_shape, dense_units=20):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=1))
    return tf.keras.Model(x, model(x))
              
critic_z = build_critic_z(
    input_shape=critic_z_input_shape,
)
critic_z.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 1)                 861       
Total params: 861
Trainable params: 861
Non-trainable params: 0
_________________________________________________________________


In [11]:
class RandomWeightedAverage(_Merge):
    def _merge_function(self, inputs):
        """
        Args:
            inputs[0] x     original input
            inputs[1] x_    predicted input
        """
        alpha = K.random_uniform((64, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

def _wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

def _gradient_penalty_loss(y_true, y_pred, averaged_samples):
    gradients = K.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    gradient_penalty = K.square(1 - gradient_l2_norm)
    return K.mean(gradient_penalty)

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [13]:
generator.trainable = False
encoder.trainable = False

x = tf.keras.Input(shape=input_shape)
y = tf.keras.Input(shape=target_shape)
z = tf.keras.Input(shape=(latent_dim, 1))

x_ = generator(z)
z_ = encoder(x)
fake_x = critic_x(x_) # Fake
valid_x = critic_x(y) # Truth

In [14]:
# interpolated_x = RandomWeightedAverage()([y, x_])
alpha = K.random_uniform((64, 1, 1))
interpolated_x = (alpha * [y, x_][0]) + ((1 - alpha) * [y, x_][1])
validity_interpolated_x = critic_x(interpolated_x)
partial_gp_loss_x = partial(_gradient_penalty_loss, averaged_samples=interpolated_x)
partial_gp_loss_x.__name__ = 'gradient_penalty'
critic_x_model = tf.keras.Model(inputs=[y, z], outputs=[valid_x, fake_x, validity_interpolated_x])
critic_x_model.compile(loss=[_wasserstein_loss, _wasserstein_loss, partial_gp_loss_x, 'sparse_categorical_crossentropy'], 
                       optimizer=optimizer, loss_weights=[1, 1, 10, 1])


    d_out_layer = Dense(1, activation='sigmoid')(fe)
    d_model = Model(in_image, d_out_layer)
    d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

2022-01-22 02:26:03.937209: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-01-22 02:26:03.973510: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f98e075c510 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-01-22 02:26:03.973529: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [ ]:
    c_out_layer = Dense(2, activation='softmax')(fe)
    c_model = Model(in_image, c_out_layer)